In [11]:
import pandas as pd
import datetime
import ffn
import cufflinks as cf
from utils import IGNIS_4H

In [12]:
# load data
ignis_4h = pd.read_csv(IGNIS_4H)
ignis_4h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 8 columns):
close         265 non-null float64
high          265 non-null float64
low           265 non-null float64
open          265 non-null float64
time          265 non-null object
volumefrom    265 non-null float64
volumeto      265 non-null float64
datetime      265 non-null object
dtypes: float64(6), object(2)
memory usage: 16.6+ KB


In [22]:
# Abnormal Returns(AR) in window period
logrets = ffn.to_log_returns(ignis_4h['close'])  # Use summable log return  
ar = ffn.to_excess_returns(logrets, 0) # Assuming market ret is 0 for simplicity

news_date = datetime.datetime(2019, 6, 5) # the date news came
window_length = 30 # window period for event study
news_idx = ignis_4h.index[pd.to_datetime(ignis_4h['datetime']) <= news_date][-1]
ar_window = ar[news_idx-window_length:news_idx+window_length] # abnormal return in window period
x_idx = pd.Series(range(-window_length,window_length)).rename("x_idx")
car = ar_window.cumsum().rename("car") # cumulative abnormal return
car = pd.concat([x_idx, car], axis=1)

In [24]:
cf.set_config_file(offline=False, world_readable=True, theme='henanigans')
car.iplot(
    x='x_idx', kind='scatter', xTitle='Time w.r.t News Release', yTitle='CAR', 
    vline={'x':0, 'color':'yellow', 'dash':'dash'},
    title='Cumulative Abnormal Return')